In [ ]:
!pip install transformers trl huggingface accelerate torch torchvision torchaudio peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.3/318.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 24.9 MB/s eta 0:00:00:00:0100:01


In [ ]:
!pip install trl

In [ ]:
!pip install accelerate

In [ ]:
!pip install -U bitsandbytes

In [ ]:
#nothing

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
import gc
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

In [ ]:
torch.cuda.empty_cache()
gc.collect()

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
model_name = "google/gemma-2-2b-it"
dataset_name = "Akshit-jain/your_very_own_therapist"
new_model = "therapist-tl"

# For LoRA
lora_r = 64  # rank
lora_alpha = 16
lora_dropout = 0.1

# Training parameters
output_dir = "./results"
num_train_epochs = 1
per_device_train_batch_size = 1
gradient_accumulation_steps = 16
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
save_steps = 0
logging_steps = 25
seq_length = 512

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!bash -i your_script.sh
!%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

bash: your_script.sh: No such file or directory
/bin/bash: line 1: fg: no job control


In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

import torch
torch.cuda.empty_cache()


In [ ]:
use_4bit = True
bnb_4bit_compute_dtype = "float16"
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

# Setup BitsAndBytes configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage=True,
)

# Prepare model for training - CRUCIAL for gradient computation
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=gradient_checkpointing)

# Configure model settings
model.config.use_cache = False
model.config.pretraining_tp = 1

# Configure LoRA with correctly set modules for Gemma
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
)

# Load dataset
dataset = load_dataset(dataset_name, split="train")

# Format dataset with proper tokenization
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=seq_length)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Training arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=False,
    bf16=torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    ddp_find_unused_parameters=False,
    dataloader_drop_last=True,
)

# Initialize trainer with the correct parameters for your version
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    peft_config=peft_config,
    args=training_arguments,
    tokenizer=tokenizer,  # This will be transformed to processing_class internally
)

# Function to monitor memory during training
def print_gpu_memory():
    if torch.cuda.is_available():
        t = torch.cuda.get_device_properties(0).total_memory
        r = torch.cuda.memory_reserved(0)
        a = torch.cuda.memory_allocated(0)
        f = r - a  # free inside reserved
        print(f"Total: {t/1e9:.2f}GB | Reserved: {r/1e9:.2f}GB | Allocated: {a/1e9:.2f}GB | Free: {f/1e9:.2f}GB")

# Print initial memory state
print_gpu_memory()

# Train
trainer.train()

# Print final memory state
print_gpu_memory()

# Save the trained model
trainer.save_model(new_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/8580 [00:00<?, ? examples/s]

<ipython-input-41-b5ebdb0777ea>:86: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Applying chat template to train dataset:   0%|          | 0/8580 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/8580 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/8580 [00:00<?, ? examples/s]

Total: 17.06GB | Reserved: 12.12GB | Allocated: 8.59GB | Free: 3.53GB


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,21.293300
50,21.039700
75,16.734200
100,20.136100
125,16.394300
150,18.750400
175,15.554200
200,19.229800
225,15.093500
250,18.955200


Total: 17.06GB | Reserved: 12.99GB | Allocated: 7.45GB | Free: 5.54GB


In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

<IPython.core.display.Javascript object>

In [ ]:
logging.set_verbosity(logging.CRITICAL)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
model.push_to_hub("Akshit-jain/therapist-ft", check_pr=True)

tokenizer.push_to_hub("Akshit-jain/therapist-ft",check_pr=True)

model.safetensors:   0%|          | 0.00/3.83G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Akshit-jain/therapist-ft/commit/537acd9f0c35b3e9bb794ec52269bfdfbc6aa344', commit_message='Upload tokenizer', commit_description='', oid='537acd9f0c35b3e9bb794ec52269bfdfbc6aa344', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Akshit-jain/therapist-ft', endpoint='https://huggingface.co', repo_type='model', repo_id='Akshit-jain/therapist-ft'), pr_revision=None, pr_num=None)

In [ ]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /kaggle/working/.git/


In [ ]:
!git remote add origin https://github.com/Aj242005/The-Therapist.git

error: remote origin already exists.


In [ ]:
!git branch -m main

In [ ]:
!git branch

In [ ]:
!git push -u origin main

error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/Aj242005/The-Therapist.git'


In [ ]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  gettext-base git-daemon-run | git-daemon-sysvinit git-doc git-email git-gui gitk gitweb git-cvs
  git-mediawiki git-svn
The following packages will be upgraded:
  git
1 upgraded, 0 newly installed, 0 to remove and 116 not upgraded.
Need to get 3,165 kB of archives.
After this operation, 4,096 B of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 git amd64 1:2.34.1-1ubuntu1.12 [3,165 kB]
Fetched 3,165 kB in 1s (4,751 kB/s)
(Reading database ... 127400 files and directories currently installed.)
Preparing to unpack .../git_1%3a2.34.1-1ubuntu1.12_amd64.deb ...
Unpacking git (1:2.34.1-1ubuntu1.12) over (1:2.34.1-1ubuntu1.11) ...
Setting up git (1:2.34.1-1ubuntu1.12) ...


In [ ]:
!git config --global user.name "Aj242005"
!git config --global user.email "akshitjain6661@gmail.com"

In [ ]:
!git add .

In [ ]:
!git status

On branch main

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   results/checkpoint-536/README.md
	new file:   results/checkpoint-536/adapter_config.json
	new file:   results/checkpoint-536/adapter_model.safetensors
	new file:   results/checkpoint-536/optimizer.pt
	new file:   results/checkpoint-536/rng_state.pth
	new file:   results/checkpoint-536/scheduler.pt
	new file:   results/checkpoint-536/special_tokens_map.json
	new file:   results/checkpoint-536/tokenizer.json
	new file:   results/checkpoint-536/tokenizer.model
	new file:   results/checkpoint-536/tokenizer_config.json
	new file:   results/checkpoint-536/trainer_state.json
	new file:   results/checkpoint-536/training_args.bin
	new file:   results/runs/Feb18_12-11-26_e7190e68a7fb/events.out.tfevents.1739880729.e7190e68a7fb.31.0
	new file:   results/runs/Feb18_12-23-01_e7190e68a7fb/events.out.tfevents.1739881411.e7190e68a7fb.31.1
	new file:   results/runs/Feb18_12-31-59_e7190e6

In [ ]:
!git commit -m "This is the main fine tuning model aspect"

[main (root-commit) 242b3c5] This is the main fine tuning model aspect
 23 files changed, 5166293 insertions(+)
 create mode 100644 results/checkpoint-536/README.md
 create mode 100644 results/checkpoint-536/adapter_config.json
 create mode 100644 results/checkpoint-536/adapter_model.safetensors
 create mode 100644 results/checkpoint-536/optimizer.pt
 create mode 100644 results/checkpoint-536/rng_state.pth
 create mode 100644 results/checkpoint-536/scheduler.pt
 create mode 100644 results/checkpoint-536/special_tokens_map.json
 create mode 100644 results/checkpoint-536/tokenizer.json
 create mode 100644 results/checkpoint-536/tokenizer.model
 create mode 100644 results/checkpoint-536/tokenizer_config.json
 create mode 100644 results/checkpoint-536/trainer_state.json
 create mode 100644 results/checkpoint-536/training_args.bin
 create mode 100644 results/runs/Feb18_12-11-26_e7190e68a7fb/events.out.tfevents.1739880729.e7190e68a7fb.31.0
 create mode 100644 results/runs/Feb18_12-23-01_e719

In [ ]:
!git push -u origin main